In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings("ignore")

In [3]:
fileNames = ['0000', '0010', '0015', '0016', '0018',
            '0028', '0030', '0036', '0048', '0050',
            '0089', '0104', '0109', '0111', '0150',
            '0256', '0406', '0518', '0519', '0583',
            '0585', '0637', '0690', '0837', '0887',
            '0893', '0958', '0999', '1006', '1088', 
            '1166', '1169', '1186', '1288', '1318',
            '1328', '1390', '1398', '1601', '1628',
            '1668', '1688', '1766', '1800', '1818',
            '1857', '1901', '1988', '1989', '1998']

rawDict = {}
featureDict = {}

for name in fileNames:
    rawDict[f'SH60{name}']=pd.read_csv(f'/Users/bkcho/Desktop/raw data/raw data 17-19/SH60{name}.csv')
    featureDict[f'SH60{name}']=pd.read_csv(f'/Users/bkcho/Desktop/engineered features/engineered features 17-19/SH60{name}.csv', index_col=0)

In [4]:
for name in fileNames:
    rawDict[f'SH60{name}']['Company']='SH60{}'.format(name)
    featureDict[f'SH60{name}']['Company']='SH60{}'.format(name)

In [5]:
rawDict['SH600010']

,Time,Open,High,Low,Close,Volume,Company
0,Day1,16.9570,17.1950,16.9570,17.0165,134513900,SH600010
1,Day2,17.0165,17.6709,16.9570,17.6114,352646700,SH600010
2,Day3,17.4329,17.6114,17.2545,17.5519,213135700,SH600010
3,Day4,17.4924,17.8494,17.3140,17.3734,148732700,SH600010
4,Day5,17.2545,18.0279,17.1355,17.8494,406777200,SH600010
...,...,...,...,...,...,...,...
751,Day752,11.1639,11.2478,10.9960,11.0800,214003568,SH600010
752,Day753,11.0800,11.0800,10.8281,10.8281,179202608,SH600010
753,Day754,10.8281,10.9960,10.8281,10.9960,95952200,SH600010
754,Day755,10.9960,10.9960,10.8281,10.9121,119114024,SH600010


In [6]:
raw = pd.DataFrame()
feature = pd.DataFrame()

for name in fileNames:
    raw=raw.append(rawDict[f'SH60{name}'])
    feature=feature.append(featureDict[f'SH60{name}'])

In [7]:
raw.shape

(37800, 7)

In [8]:
feature.shape

(37000, 12)

In [9]:
raw.head()

,Time,Open,High,Low,Close,Volume,Company
0,Day1,154.6414,159.7405,154.4560,156.9592,45036400,SH600000
1,Day2,156.9592,158.7207,156.3102,156.5883,21043100,SH600000
2,Day3,156.5883,158.5353,155.8467,158.4426,23335200,SH600000
3,Day4,158.9988,162.1510,158.9988,159.5551,33835300,SH600000
4,Day5,159.5551,160.8530,158.6280,160.0186,29530100,SH600000


In [10]:
feature.reset_index(level=0, inplace=True)
feature.rename(columns={'index':'Time'},inplace=True)
feature.head()

,Time,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,day15Returnlag,PROClag,Company
0,Day17,17860600,57.161421,0.493500,0.628701,-0.600043,132603900.0,160.05570,159.320207,0.233345,1.716987,0.058217,SH600000
1,Day18,30809600,43.371027,0.000000,0.324672,-1.000000,101794300.0,159.23984,159.202773,-3.690899,-2.282088,-2.833712,SH600000
2,Day19,14531400,42.452003,0.031911,0.175137,-0.700032,87262900.0,157.97898,158.937000,-4.745577,-3.195824,-0.239781,SH600000
3,Day20,8483500,39.311722,0.009433,0.013781,-0.894749,78779400.0,156.18038,158.510527,-3.785677,-4.287377,-0.843920,SH600000
4,Day21,22525800,34.588465,0.110347,0.050564,-0.272718,56253600.0,154.53014,157.904820,-5.238613,-5.128181,-1.463404,SH600000


In [11]:
#feature.columns=['time', 'volumelag', 'rsilag', 'fastKlag', 'fastDlag', 'ADlag', 'OBVlag', 'MA5lag', 'MA15lag', 'day5Returnlag', 'day15Returnlag', 'PROClag', 'company']

In [12]:
#raw.to_csv('raw.csv')
#feature.to_csv('feature.csv')

## Create variables

In [13]:
raw['Log_vol']=np.log(raw.Volume).replace(-np.inf, 0)

In [14]:
# y variables_Close price_% change
y_variable = []
days = [1]
for day in days:
    newcol = 'Next_' + str(day) + '_Close'
    y_variable.append(newcol)
    M = raw.groupby('Company')['Close'].shift(-day)
    N = raw.groupby('Company')['Close'].shift(0)
    raw[newcol] = (((M / N)-1) * 100)
    raw[newcol] = raw[newcol].fillna(0)

# y variables_Close price_% change
y_variable = []
days = [5,10,15]
for day in days:
    newcol = 'Next_' + str(day) + '_Close'
    y_variable.append(newcol)
    M = raw.groupby('Company')['Close'].shift(-day + 1)
    N = raw.groupby('Company')['Close'].shift(0)
    raw[newcol] = (((M / N)-1) * 100)
    raw[newcol] = raw[newcol].fillna(0)

#y variables_volume
# y_variable_v = []
# days = [1, 3, 7, 14, 30]
# for day in days:
#     newcol = 'Next_' + str(day) + '_Volume'
#     y_variable_v.append(newcol)
#     raw[newcol]=raw.groupby('Company')['Volume'].shift(-day)
#     raw[newcol] = raw[newcol].fillna(0)
    
#Relative volume
# relative_v = []
# days = [3, 7, 14, 30]
# for day in days:
#     newcol = 'Relative_' + str(day) + '_Volume'
#     relative_v.append(newcol)
#     raw[newcol]=(raw.groupby('Company')['Volume'].shift(1))/(raw.groupby('Company')['Volume'].apply(lambda x: x.shift().rolling(window=day).mean()))
#     raw[newcol] = raw[newcol].fillna(0)
    
#Relative volume_log
relative_v_log = []
days = [5, 10, 15]
for day in days:
    newcol = 'Relative_' + str(day) + '_LogVolume'
    relative_v_log.append(newcol)
    raw[newcol]=(raw.groupby('Company')['Log_vol'].shift(1))/(raw.groupby('Company')['Log_vol'].apply(lambda x: x.shift().rolling(window=day).mean()))
    raw[newcol] = raw[newcol].fillna(0)
    
#Price rate of change(ROC)
roc= []
days = [5,10,15]
for day in days:
    newcol = 'Price_rate' + str(day) + '_change'
    roc.append(newcol)
    M = raw.groupby('Company')['Close'].diff(day - 1)
    N = raw.groupby('Company')['Close'].shift(day - 1)  
    raw[newcol] = ((M / N) * 100)
    raw[newcol] = raw[newcol].fillna(0)
    
#Bollinger Upper_Bands
# upper_band= []
# days = [10,20,50]
# for day in days:
#     newcol = 'Bollinger' + str(day) + '_upperBand'
#     upper_band.append(newcol)
#     raw[newcol]=(raw.groupby('Company')['Close'].apply(lambda x: x.rolling(window=day).mean()))+(raw.groupby('Company')['Close'].apply(lambda x: x.rolling(window=day).std())*2)
#     raw[newcol] = raw[newcol].fillna(0)
    
# #Bollinger Lower_Bands
# lower_band= []
# days = [10,20,50]
# for day in days:
#     newcol = 'Bollinger' + str(day) + '_lowerBand'
#     lower_band.append(newcol)
#     raw[newcol]=(raw.groupby('Company')['Close'].apply(lambda x: x.rolling(window=day).mean()))-(raw.groupby('Company')['Close'].apply(lambda x: x.rolling(window=day).std())*2)
#     raw[newcol] = raw[newcol].fillna(0)


In [15]:
raw.shape

(37800, 18)

In [16]:
# final features
new_features=pd.merge(raw, feature, how='left',on=['Time','Company'])

In [17]:
new_features.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume','Log_vol'], inplace=True)

In [18]:
new_features=new_features.fillna(0)

## Z-scaling before feature selection

In [19]:
new_features.columns

Index(['Time', 'Company', 'Next_1_Close', 'Next_5_Close', 'Next_10_Close',
       'Next_15_Close', 'Relative_5_LogVolume', 'Relative_10_LogVolume',
       'Relative_15_LogVolume', 'Price_rate5_change', 'Price_rate10_change',
       'Price_rate15_change', 'volumelag', 'rsilag', 'fastKlag', 'fastDlag',
       'ADlag', 'OBVlag', 'MA5lag', 'MA15lag', 'day5Returnlag',
       'day15Returnlag', 'PROClag'],
      dtype='object')

In [21]:
new_features

,Time,Company,Next_1_Close,Next_5_Close,Next_10_Close,Next_15_Close,Relative_5_LogVolume,Relative_10_LogVolume,Relative_15_LogVolume,Price_rate5_change,...,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,day15Returnlag,PROClag
0,Day1,SH600000,-0.236303,1.949169,1.712929,1.417566,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
1,Day2,SH600000,1.184188,1.598587,1.065725,1.716987,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
2,Day3,SH600000,0.702147,0.351042,0.292535,-2.282088,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
3,Day4,SH600000,0.290495,-0.639152,0.755350,-3.195824,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
4,Day5,SH600000,-0.579370,0.231786,1.332595,-4.287377,0.000000,0.000000,0.000000,1.949169,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37795,Day752,SH601998,-0.487746,-0.324797,0.000000,0.000000,0.976223,0.986891,0.995823,0.653841,...,63.907078,0.868285,0.893034,-6.677966e-01,1.543553e+09,9.11500,8.901707,1.633494,3.665774,-0.161248
37796,Day753,SH601998,0.000000,0.000000,0.000000,0.000000,1.000946,1.005033,1.016057,-1.446952,...,57.247997,0.710512,0.832590,-7.769405e-01,1.520035e+09,9.12680,8.917433,0.653841,3.355827,-0.968584
37797,Day754,SH601998,-0.489030,0.000000,0.000000,0.000000,0.987725,0.988389,0.999703,-1.605739,...,54.200092,0.631447,0.736748,-6.000000e-01,1.501856e+09,9.12974,8.934140,-1.446952,2.851713,-0.488939
37798,Day755,SH601998,0.655986,0.000000,0.000000,0.000000,0.961581,0.949855,0.958730,-1.928906,...,54.200092,0.631447,0.657802,-3.010774e-14,1.501856e+09,9.10320,8.950847,-1.605739,3.025225,0.000000


## y=Next_1_Close

In [23]:
new_features.columns

Index(['Time', 'Company', 'Next_1_Close', 'Next_5_Close', 'Next_10_Close',
       'Next_15_Close', 'Relative_5_LogVolume', 'Relative_10_LogVolume',
       'Relative_15_LogVolume', 'Price_rate5_change', 'Price_rate10_change',
       'Price_rate15_change', 'volumelag', 'rsilag', 'fastKlag', 'fastDlag',
       'ADlag', 'OBVlag', 'MA5lag', 'MA15lag', 'day5Returnlag',
       'day15Returnlag', 'PROClag'],
      dtype='object')

In [27]:
Next_1_Close_y=new_features.iloc[:,6:]

In [28]:
Next_1_Close_y['Next_1_Close']=new_features['Next_1_Close']

In [29]:
Next_1_Close_y.to_csv('Next_1_Close_y.csv')

## y=Next_5_Close

In [30]:
Next_5_Close_y=new_features.iloc[:,6:]

In [31]:
Next_5_Close_y['Next_5_Close']=new_features['Next_5_Close']

In [32]:
Next_5_Close_y.to_csv('Next_5_Close_y.csv')

## y=Next_10_Close

In [33]:
Next_10_Close_y=new_features.iloc[:,6:]

In [34]:
Next_10_Close_y['Next_10_Close']=new_features['Next_10_Close']

In [35]:
Next_10_Close_y.to_csv('Next_10_Close_y.csv')

## y=Next_15_Close

In [36]:
Next_15_Close_y=new_features.iloc[:,6:]

In [37]:
Next_15_Close_y['Next_15_Close']=new_features['Next_15_Close']

In [38]:
Next_15_Close_y.to_csv('Next_15_Close_y.csv')